In [1]:
# what genes experienced putative gene family collapse?

In [40]:
import pandas as pd
from tools.sqlInterface import *
from collections import *

In [7]:
df = load_filter_evaluation('/public/groups/cgl/cat/primates_evan/out/databases/Rhesus.db')

In [9]:
ref_df = load_annotation('/public/groups/cgl/cat/primates_evan/out/databases/Human.db')

In [20]:
merged = df.merge(ref_df, on=['GeneId', 'TranscriptId'])

In [21]:
merged_collapsed = merged[~merged.CollapsedGeneIds.isnull()]

In [71]:
r = []
# this was actually calculated per-gene in the original analysis, so we can just look at the first transcript
for (gene_biotype, gene_id, gene_name), d in merged_collapsed.groupby(['GeneBiotype', 'GeneId', 'GeneName']):
    d = d.iloc[0]
    r.append([gene_id, gene_name, gene_biotype, d.CollapsedGeneIds.count(',') + 1, d.CollapsedGeneIds, d.CollapsedGeneNames])
collapsed_df = pd.DataFrame(r, columns=['GeneId', 'GeneName', 'GeneBiotype', 'NumberOfCollapsedCopies', 'CollapsedGeneIds', 'CollapsedGeneNames'])

In [72]:
c = collapsed_df[collapsed_df.GeneBiotype == 'protein_coding']
Counter(c.NumberOfCollapsedCopies), len(c)

(Counter({2: 295,
          1: 473,
          3: 28,
          4: 17,
          7: 1,
          17: 1,
          5: 7,
          6: 1,
          8: 2,
          14: 1,
          21: 1}),
 827)

In [78]:
# split gene analysis
# this was done per-transcript, so the intervals may vary per transcript
split = merged[~merged.PossibleSplitGeneLocations.isnull()]
split = split[['GeneId', 'GeneName', 'TranscriptId', 'PossibleSplitGeneLocations']].sort_values(['GeneId', 'GeneName'])
split.columns = ['Gene ID', 'Gene Name', 'Transcript ID', 'Locations of split mappings']
print(len(set(split['Gene ID'])))

550


In [92]:
# this was done per gene
paralogy = merged[~merged.GeneAlternateLoci.isnull()]
paralogy = paralogy[['GeneId', 'GeneName', 'GeneBiotype', 'GeneAlternateLoci']].groupby('GeneId').first().reset_index()
paralogy.columns = ['Gene ID', 'Gene Name', 'Gene Biotype', 'Loci of possible paralogous mappings']
paralogy['Number of possible paralogous mappings'] = [x.count(',') + 1 for x in paralogy['Loci of possible paralogous mappings']]
paralogy = paralogy.sort_values(['Gene Biotype', 'Gene ID', 'Gene Name'])
Counter(paralogy['Number of possible paralogous mappings']), len(paralogy)

(Counter({2: 571,
          3: 267,
          1: 2512,
          4: 108,
          6: 67,
          8: 32,
          11: 16,
          9: 18,
          5: 86,
          17: 9,
          7: 44,
          14: 7,
          10: 9,
          13: 7,
          24: 2,
          34: 1,
          12: 10,
          30: 1,
          18: 1,
          20: 2,
          16: 3,
          21: 3,
          22: 2,
          43: 1,
          15: 3,
          26: 3,
          23: 1}),
 3786)

In [93]:
paralogy_coding = paralogy[paralogy['Gene Biotype'] == 'protein_coding']
Counter(paralogy_coding['Number of possible paralogous mappings']), len(paralogy_coding)

(Counter({1: 711,
          2: 116,
          3: 62,
          7: 18,
          5: 13,
          43: 1,
          17: 6,
          14: 1,
          8: 9,
          22: 1,
          6: 8,
          4: 16,
          15: 2,
          26: 1,
          9: 2}),
 967)

In [ ]:
with pd.ExcelWriter('rhesus_transMap_metrics.xlsx') as fh:
    collapsed_df.to_excel(fh, sheet_name='Gene Family Collapse')
    split.to_excel(fh, sheet_name='Split Gene Analysis')
    paralogy.to_excel(fh, sheet_name='Putative Paralogous Loci')